<a href="https://colab.research.google.com/github/GerardoToso/Competencia-FSadosky-e-YPF-/blob/main/01_Analisis_exploratorio_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="float:left; border:none">
   <tr style="border:none">
       <td style="border:none">  
           <img 
               src="assets/Logo_de_YPF.png" 
               style="width:300px"
           >
           </a>    
       </td>
   </tr>
</table>
<div style="float:right;"><h2>01. Análisis exploratorio de datos</h2></div>

 <a id="top"></a> <br>
## Índice
1. [Presentación del problema](#1)<br>
2. [Carga de paquetes y datos](#2)<br>
    2.1. [Importación de librerias](#21)<br>
    2.2. [Lectura de datos](#22)<br>
3. [Analisis exploratorio de datos](#3)<br>
    3.1. [Primer aproximamiento a los datos](#31)<br>
    3.2. [Analisis de valores nulos](#32)<br>
    3.3. [Análisis de variable target](#33)<br>
    3.4. [Análisis de variable City](#34)<br>
    3.5. [Análisis de variable City Development Index](#35)<br>
    3.6. [Análisis de variable Education Level](#36)<br>
    3.7. [Análisis de variable Company Type y Company Size](#37)<br>
    3.8. [Análisis de variable Training hours](#38)<br>

***
<a id="1"></a> <br>
## 1. Presentación del problema

***
<a id="2"></a> <br>
## 2. Carga de paquetes y datos

<a id="21"></a>
### 2.1. Importación de librerias

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
import warnings
import plotly.express as px
import chardet

<a id="22"></a>
### 2.2. Lectura de datos

In [ ]:
# Detección automática de la codificación de los caracteres
with open("../data/Dataset participantes.csv", 'rb') as f:
    enc = chardet.detect(f.read())  # or readline if the file is large

# Lectura del csv
df = pd.read_csv("../data/Dataset participantes.csv", encoding = enc['encoding'],sep='\t')

FileNotFoundError: ignored

La tabla contiene las columnas que se describen en la siguiente lista:

- ID_FILA: Identificador usado para subir la solución.
- ID_EVENTO: Identificador único de la línea.
- CAMPO: Nombre del Bloque al que pertenece el PAD Hijo.
- FLUIDO: Fluido que produce el PAD (Gas o Petróleo).
- PAD_HIJO: Nombre del PAD Hijo: HIJO: Nombre del Pozo Hijo.
- ETAPA_HIJO: Número de Etapa de Fractura del pozo Hijo.
- PADRE: Nombre del Pozo Padre.
- D3D: Distancia tridimensional entre la etapa de fractura del Pozo Hijo y la etapa de fractura más cercana del Pozo Padre (DX^2+DY^2+DZ^2)/2.
- D2D: Distancia en planta (vista de pajaro - sin considerar distancia vertical) entre la etapa de fractura del Pozo Hijo y la etapa de fractura más cercana del Pozo Padre (DX^2+DY^2)*(1/2).
- DZ: Distancia vertical (diferencia de altura) entre la etapa de fractura del Pozo Hijo y la etapa de fractura más cercana del Pozo Padre.
- AZ: Angulo respecto al norte que forma la linea recta que une la etapa de fractura del Pozo Hijo y la etapa de fractura más cercana del Pozo Padre.
- #_BARRERAS: Cantidad de pozos padre entre el Pozo Hijo y el Pozo Padre considerado. Se consideran todos los pozos atravesados por la linea considerada en la D2D, y dentro de un espacio vertical que abarca toda la distancia vertical entre Pozo Hijo y Pozo Padre más 40m por encima y por debajo.
- LINEAMIENTO: Anomalías geológicas que atraviesan al Pozo Hijo y Pozo Padre.
- WHP_i: Presión al inicio de la interferencia.
- delta_WHP: Salto de presión generado por la interferencia. Desde la presión inicial al punto máximo o mínimo del salto.
- ESTADO: Estado del pozo al momento de la interferencia. Cerrado/Abierto.

Es importante notar que las variables WHP_i y delta_WHP corresponden respectivamente a P y ΔP de la ecuación 1. E s DP la variable que debe estimarse con el modelo cuyo salida será ΔPest de la ecuación. En dicha tabla cada fila corresponderá a un evento a modelar.

***
<a id="3"></a> <br>
## 3. Analisis exploratorio de datos

<a id="31"></a>
### 3.1. Primer aproximamiento a los datos

In [ ]:
# Se genera un reporte con pandas profiling
data_report = ProfileReport(df)
data_report

In [ ]:
hajjhsjhvkj